### Generate street name and district mapping
- We will generate a mapping of street names to districts using the dataset "Danh sách tên các con đường tại thành phố Đà Nẵng" [1]

In [1]:
import pandas as pd
import re
import numpy as np

df = pd.read_excel("../input_data/danh_sach_ten_cac_con_duong_tai_thanh_pho_da_nang_.xlsx")

def extract_quan(mo_ta):
    if not isinstance(mo_ta, str):
        return None
    match = re.search(r"quận[:\s]*([^\n\.,\d]*)", mo_ta, re.IGNORECASE)
    if match:
        return match.group(1).strip()
    matches = re.findall(r"quận[:\s]*([^\n\.,\d]*)", mo_ta, re.IGNORECASE)
    districts = [m.strip() for m in matches if m.strip()]
    if districts:
        return ", ".join(districts)
    # Nếu không tìm thấy, thử regex khác
    matches = re.findall(r"thuộc quận[:\s]*([^\n\.,\d]*)", mo_ta, re.IGNORECASE)
    districts = [m.strip() for m in matches if m.strip()]
    if districts:
        return ", ".join(districts)
    # Thử thêm một regex nữa nếu vẫn chưa có
    matches = re.findall(r"phường[:\s]*[^\n\.,\d]*,\s*quận[:\s]*([^\n\.,\d]*)", mo_ta, re.IGNORECASE)
    districts = [m.strip() for m in matches if m.strip()]
    if districts:
        return ", ".join(districts)
    return None


df["district"] = df["Mô tả"].apply(extract_quan)
df["street"] = df["Tên đường"]
df["district"] = (
    df["district"]
    .astype(str)
    .str.replace(r"\s+", " ", regex=True)           # Replace multiple spaces with one
    .str.strip()                                    # Remove leading/trailing spaces
    .str.replace(r"[^a-zA-ZÀ-ỹà-ỹ0-9\s]", "", regex=True)  # Remove non-alphanumeric (keep Vietnamese chars)
)
df["street"] = df["street"].astype(str).str.strip()
df = df.drop_duplicates(subset=["street"], keep="first")


### Random with roulette wheel selection
- We will use a random selection method with roulette wheel selection to choose a street name and district from the generated mapping.

In [2]:
valid_districts = df["district"][df["district"].notnull() & (df["district"].str.strip() != "") & (df["district"].str.strip() != "None")]

district_counts = valid_districts.value_counts(normalize=True)

mask_missing = df["district"].isnull() | (df["district"].str.strip() == "") | (df["district"].str.strip() == "None")

np.random.seed(42)

random_districts = np.random.choice(
    district_counts.index,
    size=mask_missing.sum(),
    p=district_counts.values
)

df.loc[mask_missing, "district"] = random_districts


### Comments: The output data still contains rows such as "Nguyễn Triệu Luật, Ngũ Hành Sơn vào năm", which indicates that further normalization is needed.
- We will use dynamic programming to normalize the district names.

In [3]:
from algorithms import StringComparison
# Re-read the Excel file to get candidate district names for normalization (assumes the file contains columns "Tỉnh / Thành Phố" and "Tên")
df_candidate = pd.read_excel("../input_data/DanhSachCapHuyen14_05_2025.xlsx")

# Filter by city "Thành phố Đà Nẵng"
df_candidate = df_candidate[df_candidate["Tỉnh / Thành Phố"] == "Thành phố Đà Nẵng"]

# Get unique candidate names from the "Tên" column
candidate_districts = df_candidate["Tên"].dropna().unique()

# Normalize the district column in df using candidate names if similarity >= 0.7
def normalize_district(district):
    best_match = district
    best_ratio = 0.0
    for cand in candidate_districts:
        comparison = StringComparison(district.lower(), str(cand).lower())
        ratio = comparison.similarity()
        if ratio >= 0.7 and ratio > best_ratio:
            best_match = cand
            best_ratio = ratio
    return best_match

df["district"] = df["district"].apply(normalize_district)
df["district"] = df["district"].str.replace(r"\bQuận\s*", "", regex=True)

In [4]:

df[["street", "district"]].to_csv("../processed_data/street_district.csv", index=False)
print("Exported district and street to processed_data/street_district.csv")

Exported district and street to processed_data/street_district.csv


In [6]:
from embeddings import TFIDFEmbedding, CharNgramEmbedding

combined = list(set(df["street"].values).union(set(df["district"].values)))
print("combined", (combined))
# embedding = TFIDFEmbedding(combined)
embedding = CharNgramEmbedding(combined)
embedding.export("../processed_data/chargram_embedding.pkl")

combined ['Trung Nghĩa 4', 'Hoài Thanh', 'Hòa Minh 6', 'Mỹ Đa Tây 8', 'Tân Lưu', 'Tôn Đản', 'Xuân Thiều 34', 'Hòa Bình 5', 'Lê Hữu Khánh', 'Lỗ Giáng 11', 'Phong Bắc 7', 'Trần Hữu Duẩn', 'Trần Bích San', 'Quán Khải 8', 'Nguyễn Văn Cừ', 'An Thượng 18', 'Hoàng Minh Thảo', 'Hoàng Tích Trí', 'Cẩm Chánh 2', 'Phong Bắc', 'Nam Trân', 'Sơn Thủy 5', 'Lê Khắc Cần', 'Trung Nghĩa 6', 'Đức Lợi 2', 'Lê Đình Chinh', 'Lương Định Của', 'Hoàng Văn Thụ', 'Tiên Sơn 17', 'Tống Duy Tân', 'Thanh Vinh 6', 'Nguyễn Chu Sĩ', 'Liêm Lạc 5', 'Phong Bắc 5', 'Kim Đồng', 'Đa Mặn 5', 'Bùi Xương Trạch', 'Nguyễn Văn Huề', 'Thích Phước Huệ', 'Hòa Phú 22', 'Lương Thúc Kỳ', 'Bá Giáng 11', 'Hàm Trung 5', 'Thanh Huy 3', 'Hoàng Ngân', 'Nhơn Hòa 6', 'Phú Lộc 3', 'Hòa An 7', 'Vũ Văn Cẩn', 'An Hoà 7', 'Phan Huy Chú', 'Hồ Sỹ Đống', 'Nguyễn Xuân Nhĩ', 'Cao Sơn 3', 'An Hải 12', 'An Bắc 3', 'Tiên Sơn 15', 'An Thượng 3', 'Vùng Trung 1', 'Suối Đá 2', 'Đông Hải 14', 'Nguyễn Khoa Chiêm', 'Nguyễn Chu Sỹ', 'Đào Duy Kỳ', 'Thuỷ Sơn 4', 'Non N